# Procesamiento de Información 2023
## Unidad 5 - Tarea
## Modelado de espacio vectorial
### David Aarón Ramírez Olmeda

## Introducción

Para abordar esta tarea, podemos aprovechar gran parte del trabajo realizado en la actividad anterior y realizar algunas modificaciones para incorporar la creación de vectores y el uso de la similitud de coseno.

Detectar el plagio manualmente puede ser una tarea tediosa y poco efectiva, por lo que es importante contar con herramientas automatizadas para realizar esta tarea. El procesamiento del lenguaje natural y los algoritmos de similitud coseno son técnicas que pueden ser útiles para resolver este problema.

Teniendo en cuenta estos conceptos, podemos aplicar lo aprendido en la lección sobre la medida coseno. Esta medida se utiliza comúnmente con vectores que contienen valores reales que definen diferentes ponderaciones de las características del objeto. Principalmente se utiliza para comparar objetos durante los procesos de agrupamiento o clasificación de datos. La fórmula para calcular la medida coseno es la siguiente:

$$
Coseno(\vec{x}, \vec{y}) = \frac{\vec{x} \cdot \vec{y}}{\|\vec{x}\| \|\vec{y}\|} = \frac{\sum_{i=1}^N x_i \times y_i}{\sqrt{\sum_{i=1}^N x_i^2} \sqrt{\sum_{i=1}^N y_i^2}}
$$


## Desarrollo

A grandes rasgos:

- Se carga la muestra de archivos fuente y los archivos sospechosos.
- Se preprocesan los archivos fuente y los archivos sospechosos.
- Se crea una matriz de similitud coseno entre la muestra de archivos fuente y los archivos sospechosos.
- Se encuentran los 3 archivos más parecidos para cada uno de los 20 archivos de la muestra de archivos fuente.

In [1]:
import os
import random
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import numpy as np
from collections import defaultdict
from itertools import combinations
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
#nltk.download("stopwords")
stop_words = set(stopwords.words("english"))
stemmer = SnowballStemmer("english")
#nltk.download('punkt')

In [3]:
def load_files(directory):
    files = []
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            filepath = os.path.join(directory, filename)
            with open(filepath, "r", encoding="utf-8") as f:
                text = f.read()
                files.append((filename, text))
    return files

In [4]:
source_dir = "source-documents"
suspicious_dir = "suspicious-documents"

muestra = 20
source_files = load_files("source-documents")
source_sample = random.sample(source_files, muestra)
suspicious_files = load_files("suspicious-documents")

In [5]:
print("Los 20 archivos muestra seleccionados serán:")
for filename, text in source_sample:
    print(filename)

Los 20 archivos muestra seleccionados serán:
source-document0056.txt
source-document0034.txt
source-document0050.txt
source-document0014.txt
source-document0041.txt
source-document0074.txt
source-document0151.txt
source-document0148.txt
source-document0230.txt
source-document0236.txt
source-document0168.txt
source-document0208.txt
source-document0212.txt
source-document0207.txt
source-document0054.txt
source-document0189.txt
source-document0109.txt
source-document0192.txt
source-document0198.txt
source-document0166.txt


In [6]:
# Preprocesamiento
def preprocess(text):
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = text.lower()
    tokens = nltk.word_tokenize(text)
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [stemmer.stem(token) for token in tokens]
    return " ".join(tokens)

In [7]:
# Creación de vectores TF-IDF
corpus = [preprocess(text) for filename, text in source_sample + suspicious_files]
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(corpus)

# Cálculo de similitud coseno
similarity_matrix = cosine_similarity(tfidf[:muestra], tfidf[muestra:])

TfidfVectorizer proporciona una forma fácil y eficiente de convertir los textos en vectores TF-IDF y de calcular la similitud coseno entre los vectores.

In [8]:
# Encontrar los 3 archivos más parecidos para la muestra (para cada archivo de la muestra)
for i, file in enumerate(source_sample):
    similarities = similarity_matrix[i]
    top_similar_indices = np.argsort(similarities)[-3:][::-1]
    print("({}) Archivo de la muestra:".format(i+1), file[0])
    print("Texto original del archivo de muestra:\n", file[1][:100], "...")
    for j, index in enumerate(top_similar_indices):
        suspicious_file = suspicious_files[index][0]
        similarity = similarities[index]
        print("- - - Archivo sospechoso #{}:".format(j+1), suspicious_file, " - Similitud:", similarity)
        print("Texto original del archivo sospechoso:\n", suspicious_files[index][1][:100], "...")
    print()

(1) Archivo de la muestra: source-document0056.txt
Texto original del archivo de muestra:
 But the lesson that should not be lost is the transcendent one: Clarence Thomas made it in America b ...
- - - Archivo sospechoso #1: suspicious-document0560.txt  - Similitud: 0.16717423611014134
Texto original del archivo sospechoso:
 It's time, once again, for the screaming.  Evan O'Neal , 4, ambles into an examination room at the   ...
- - - Archivo sospechoso #2: suspicious-document2107.txt  - Similitud: 0.14989602829196094
Texto original del archivo sospechoso:
  WASHINGTON  _ A federal judge  Monday  found President  Clinton  in civil contempt of court for lyi ...
- - - Archivo sospechoso #3: suspicious-document0559.txt  - Similitud: 0.1469912884752169
Texto original del archivo sospechoso:
 Was  TJ Maxx  trying to promote a more youthful corporate culture when a young manager at a  New Jer ...

(2) Archivo de la muestra: source-document0034.txt
Texto original del archivo de muestra:
 Firef

## Conclusión

Hemos usado la técnica de similitud coseno para detectar plagio en textos. Al utilizar esta medida y la creación de vectores TF-IDF, se pueden encontrar similitudes entre diferentes documentos y determinar si existe plagio o no.